**Speech Denoising Question**

In [1]:
!pip install librosa # in colab, you'll need to install this
import librosa

In [0]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

import matplotlib.pyplot as plt
import pylab as pl
import pandas as pd
import numpy as np

from IPython.display import display, clear_output
from __future__ import print_function
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets
from math import ceil
from IPython.display import Audio
from scipy.io import wavfile
import math

%matplotlib inline

In [0]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
from google.colab import files
uploaded = files.upload()

Saving test_x_01.wav to test_x_01 (5).wav
Saving test_x_02.wav to test_x_02 (5).wav
Saving train_clean_male.wav to train_clean_male (5).wav
Saving train_dirty_male.wav to train_dirty_male (5).wav


In [5]:
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

User uploaded file "test_x_01.wav" with length 145284 bytes
User uploaded file "test_x_02.wav" with length 388752 bytes
User uploaded file "train_clean_male.wav" with length 2522886 bytes
User uploaded file "train_dirty_male.wav" with length 2522898 bytes


In [0]:
s, sr=librosa.load('train_clean_male.wav', sr=None)
S_inp=librosa.stft(s, n_fft=1024, hop_length=512)
sn, sr=librosa.load('train_dirty_male.wav', sr=None)
X_inp=librosa.stft(sn, n_fft=1024, hop_length=512)
sx1, sr=librosa.load('test_x_01.wav', sr=None)
test1 = librosa.stft(sx1, n_fft=1024, hop_length=512)
sx2, sr=librosa.load('test_x_02.wav', sr=None)
test2 = librosa.stft(sx2, n_fft=1024, hop_length=512)

In [7]:
S_inp.shape
X_inp.shape
test1.shape
test2.shape

(513, 2459)

(513, 2459)

(513, 142)

(513, 380)

In [8]:
#Getting magnitudes as S and X are complex valued

S_mod = np.transpose(np.abs(S_inp))
X_mod = np.transpose(np.abs(X_inp))
test1_mod = np.transpose(np.abs(test1))
test2_mod = np.transpose(np.abs(test2))

S_mod.shape
X_mod.shape
test1_mod.shape
test2_mod.shape

(2459, 513)

(2459, 513)

(142, 513)

(380, 513)

**Deep Neural Network for Speech Denoising**

In [0]:
#Training and running neural net

#Weight Variable
def weights(shape, name):
    i = tf.contrib.layers.variance_scaling_initializer(factor=2.0 , mode='FAN_AVG',uniform=True, seed=None, dtype=tf.float32)
    return tf.Variable(i(shape), name=name)

#Bias Variable
def bias(shape, name):
    i = tf.contrib.layers.variance_scaling_initializer(factor=2.0 , mode='FAN_AVG',uniform=True, seed=None, dtype=tf.float32)
    return tf.Variable(i(shape), name=name)

In [0]:
X = tf.placeholder(tf.float32, [None, 513], name="X")
y = tf.placeholder(tf.float32, [None, 513], name="y")
dropout_var = tf.placeholder("float")

def neural_net(X, dp):
    # Input Layer with 513 units
    weights_1 = weights([513, 513], "weights_1")
    bias_1 = bias([513], "bias_1")

    layer_1 = tf.nn.dropout(tf.nn.relu(tf.matmul(X, weights_1) + bias_1), keep_prob = dp)

    #First Hidden Layer with 1024 units
    weights_2 = weights([513, 1024], "weights_2")
    bias_2 = bias([1024], "bias_2")

    layer_2 = tf.nn.dropout(tf.nn.relu(tf.matmul(layer_1, weights_2) + bias_2), keep_prob = dp)

    #Second Hidden Layer with 1024 units
    weights_3 = weights([1024, 1024], "weights_3")
    bias_3 = bias([1024], "bias_3")

    layer_3 = tf.nn.dropout(tf.nn.relu(tf.matmul(layer_2, weights_3) + bias_3), keep_prob = dp)

    #Third Hidden Layer with 1024 units
    weights_4 = weights([1024, 1024], "weights_4")
    bias_4 = bias([1024], "bias_4")

    layer_4 = tf.nn.dropout(tf.nn.relu(tf.matmul(layer_3, weights_4) + bias_4), keep_prob = dp)

    #Fourth Hidden Layer with 1024 units
    weights_5 = weights([1024, 1024], "weights_5")
    bias_5 = bias([1024], "bias_5")

    layer_5 = tf.nn.dropout(tf.nn.relu(tf.matmul(layer_4, weights_5) + bias_5), keep_prob = dp)

    #Output layer with 513 units
    output_weights = weights([1024, 513], "output_weights")
    output_bias = bias([513], "output_bias")

    output = tf.nn.relu(tf.matmul(layer_5, output_weights) + output_bias)

    return ([output, layer_5, layer_4, layer_3, layer_2, layer_1])

#Output 
out = neural_net(X, dropout_var)
output = out[0]

#Optimization
cost = tf.reduce_mean(tf.losses.mean_squared_error(output, y)) 
train_optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

#Initializer
init = tf.global_variables_initializer()

In [0]:
#Function to calculate SNR

def SNR_function(s, s_pred):
    
    nlen = min(len(s), len(s_pred))
    SNR = 10*math.log10((np.sum(s[:nlen]**2))/(np.sum((s[:nlen] - s_pred[:nlen])**2)))
    
    return SNR

In [42]:
#Running model on training speech signal dataset and checking

max_epochs = 570
batch_size = 100
step = 5

sess = tf.Session()
sess.run(init)

for epoch in range(max_epochs):
    avg_cost = 0.
    random = np.arange(0, 2459, 100)
    np.random.shuffle(random)
    
    for i in range(len(random)):
        start = int(random[i])
        end = int(start + batch_size)
        b_x, b_y = np.array(X_mod[start:end,:]), np.array(S_mod[start:end])
        data = {X: b_x, y: b_y, dropout_var : 0.7}
        sess.run(train_optimizer, feed_dict=data)
        avg_cost += sess.run(cost, feed_dict=data)
     
    avg_cost = avg_cost / len(random)
        
    if (epoch+1) % step == 0:
        print ("Epoch: %03d/%03d cost: %.9f" % (epoch, max_epochs, avg_cost))
        data = {X: b_x, y: b_y, dropout_var : 1.}
        train_out = sess.run(out, feed_dict=data)
        train_output = sess.run(output, feed_dict=data)
        
        data = {X: X_mod, y: S_mod, dropout_var : 1.}
        full_train_out = sess.run(out, feed_dict=data)
        full_train_output = sess.run(output, feed_dict=data)
        
print ("=========================Model Optimization Complete============================")

Epoch: 004/570 cost: 0.044556595
Epoch: 009/570 cost: 0.034643507
Epoch: 014/570 cost: 0.035646060
Epoch: 019/570 cost: 0.034003653
Epoch: 024/570 cost: 0.031509107
Epoch: 029/570 cost: 0.030037047
Epoch: 034/570 cost: 0.029622766
Epoch: 039/570 cost: 0.029048664
Epoch: 044/570 cost: 0.032490938
Epoch: 049/570 cost: 0.028431777
Epoch: 054/570 cost: 0.028496795
Epoch: 059/570 cost: 0.025769016
Epoch: 064/570 cost: 0.027836792
Epoch: 069/570 cost: 0.026764888
Epoch: 074/570 cost: 0.026750831
Epoch: 079/570 cost: 0.025101205
Epoch: 084/570 cost: 0.028017273
Epoch: 089/570 cost: 0.025363080
Epoch: 094/570 cost: 0.023924638
Epoch: 099/570 cost: 0.026550242
Epoch: 104/570 cost: 0.031202120
Epoch: 109/570 cost: 0.027349616
Epoch: 114/570 cost: 0.024495057
Epoch: 119/570 cost: 0.024037170
Epoch: 124/570 cost: 0.024821224
Epoch: 129/570 cost: 0.022245292
Epoch: 134/570 cost: 0.021758784
Epoch: 139/570 cost: 0.021557602
Epoch: 144/570 cost: 0.020991417
Epoch: 149/570 cost: 0.020321552
Epoch: 154

In [43]:
#SNR Calculation on whole dataset
s_hat = (X_inp / np.abs(X_inp)) * np.transpose(full_train_output)
s_pred = librosa.istft(s_hat, win_length = 1024, hop_length = 512)

s.shape
s_pred.shape

print (SNR_function(s,s_pred))

(1258899,)

(1258496,)

12.481092631062156


In [0]:
librosa.output.write_wav('train_recovered.wav', s_pred, sr)

files.download('train_recovered.wav')

**Getting output on Test datasets - 1 and 2**

In [0]:
#On test signal 1

data = {X: test1_mod, y : S_mod, dropout_var : 1.}
test1_out = sess.run(out, feed_dict=data)
test1_output = sess.run(output, feed_dict=data)

test1_hat = (test1 / np.abs(test1)) * np.transpose(test1_output)
test1_pred = librosa.istft(test1_hat, win_length = 1024, hop_length = 512)


librosa.output.write_wav('test_s_01_recons.wav', test1_pred, sr)

files.download('test_s_01_recons.wav')

In [0]:
#On test signal 2

data = {X: test2_mod, y : S_mod, dropout_var : 1.}
test2_out = sess.run(out, feed_dict=data)
test2_output = sess.run(output, feed_dict=data)

test2_hat = (test2 / np.abs(test2)) * np.transpose(test2_output)
test2_pred = librosa.istft(test2_hat, win_length = 1024, hop_length = 512)


librosa.output.write_wav('test_s_02_recons.wav', test2_pred, sr)

files.download('test_s_02_recons.wav')